# Code to get tracks set up for CoVid-relevant regulatory architecture
# Lung (epithelial-like) regulatory architecture

# Minimal tracks to prepare 
Track : format - source

    * CTCF : bigwig - Imputed CD8 T-cell ENCODE 
    * H3K27Ac : bigwig - ENCODE NHLF
    * H3k4Me3 : bigwig - ENCODE SAEC
    * H3k9Me3 : bigwig - ENCODE NHLF
    * DNAse-seq : bigwig - ENCODE SAEC
    * Methylation : bigwig - None
    * Loops : links - hESC
    * Hi-C : cool - IMR90 GItar
    * Genes : genes_bed -  Gencode
    * Repeats : bed - L1Base2
    * Chromatin state : bed - ENCODE/Segway
    * eQTL list : arcs - none
    * GWAS : bed - pvals and bigwig Ellinghaus and Covid19Hg
    * RNA-Seq : bigwig and txt - HBEC, BALF, BALF-Covid
    

In [ ]:
import pyensembl, os, sys, re, numpy as np
from helper_funcs import *


### Transcription factor and histones

In [ ]:
%%bash
#Get Histone marks:

# Get bigwig H3k4me3 fold change over control 
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF782BOR_H3k4Me3_SAEC.bw --quiet \
    "https://encode-public.s3.amazonaws.com/2016/08/23/89ae3a85-af02-49bd-b378-e565595106b4/ENCFF782BOR.bigWig"

# Get bigwig H3K27Ac fold change over control
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF637KNN_H3k27Ac_NHLF.bw --quiet \
    "https://encode-public.s3.amazonaws.com/2017/02/07/b8c9384f-af28-48bd-9474-2293237bba68/ENCFF637KNN.bigWig"    

# Get bigwig H3K9Me3 fold change over control
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF143LCY_H3k9Me3_NHLF.bw --quiet \
    "https://encode-public.s3.amazonaws.com/2016/11/28/77fa3756-baeb-4a1b-967b-d2d02836db92/ENCFF143LCY.bigWig"


In [ ]:
%%bash
#Get Bigwigs:

# Get CTCF chip-seq fold change over control
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF117BRM_CTCF_NHLF.bw --quiet \
    "https://encode-public.s3.amazonaws.com/2016/12/16/6f8d8132-8da6-467e-a4e3-5f720ae2370c/ENCFF117BRM.bigWig"

# Get called CTCF peaks
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF777ODE_CTCF_NHLF.bed.gz --quiet \
    "https://encode-public.s3.amazonaws.com/2016/12/16/b9ad65ca-eb0d-4c6e-8d38-64346569f026/ENCFF777ODE.bed.gz"
gunzip /input_dir/corona_analysis/tracks/ENCFF777ODE_CTCF_NHLF.bed.gz
    

### Chromatin accessibility

In [ ]:
%%bash

# Get bigwig DNAse normalized read count 
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF827VFY_DNAse_SAEC.bw --quiet \
"https://encode-public.s3.amazonaws.com/2017/08/24/65b36dd7-fa4e-47ba-91a8-536ebcff4d93/ENCFF827VFY.bigWig"


In [ ]:
%%bash

# Get bed narrowpeak DNAse 
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF827VFY_DNAse_SAEC.bed.gz --quiet \
"https://encode-public.s3.amazonaws.com/2018/05/14/0b8ae8d1-bac1-41b2-b8fd-7795544fcc19/ENCFF546QUZ.bed.gz"
gunzip /input_dir/corona_analysis/tracks/ENCFF827VFY_DNAse_SAEC.bed.gz


### Chromatin looping

In [ ]:
#Got loops from previous hESC paper in:
# /input_dir/corona_analysis/tracks/primed_.7_origami_intra.arcs

In [ ]:
%%bash

#Get Hi-C contact matrix for IMR90 from GenomeGitar
wget -O /input_dir/corona_analysis/tracks/GSM1551599_contact_matrices.zip --quiet -nc \
    https://data.genomegitar.org/GSM1551599/GSM1551599_contact_matrices.zip
unzip /input_dir/corona_analysis/tracks/GSM1551599_contact_matrices.zip

#Get Hi-C data in h5 for IMR90 from GenomeGitar
wget -O /input_dir/corona_analysis/tracks/GSM1551599_hdf5.zip --quiet -nc \
    https://data.genomegitar.org/GSM1551599/GSM1551599_hdf5.zip
unzip /input_dir/corona_analysis/tracks/GSM1551599_hdf5.zip


In [ ]:
%%bash

#Get Hi-C data from 4DN -- IMR90 In-situ Hi-C same as above
https://data.4dnucleome.org/experiment-set-replicates/4DNES1ZEJNRU/
    https://data.4dnucleome.org/files-processed/4DNFIH7TH4MF/@@download/4DNFIH7TH4MF.hic
    

### Chromatin state

In [ ]:
%%bash

#Get fetal kidney chromatin state as predicted by Segway HMM model
wget --quiet -nc https://noble.gs.washington.edu/proj/encyclopedia/interpreted/BC_LUNG_01-11002.bed.gz 
gunzip BC_LUNG_01-11002.bed.gz 
sort-bed BC_LUNG_01-11002.bed | bgzip > /input_dir/corona_analysis/tracks/Segway_bc_lung.bed.gz
rm BC_LUNG_01-11002.bed
tabix -p bed /input_dir/corona_analysis/tracks/Segway_bc_lung.bed.gz


In [ ]:
%%bash
#Get NHLF ChromHmm model
wget --quiet -nc -O E128_15_coreMarks_dense.bed.gz \
    https://egg2.wustl.edu/roadmap/data/byFileType/chromhmmSegmentations/ChmmModels/coreMarks/jointModel/final/E128_15_coreMarks_dense.bed.gz
gunzip E128_15_coreMarks_dense.bed.gz
sort-bed E128_15_coreMarks_dense.bed | bgzip > /input_dir/corona_analysis/tracks/E128_15_coreMarks_dense.bed.gz
rm E128_15_coreMarks_dense.bed
tabix -p bed /input_dir/corona_analysis/tracks/E128_15_coreMarks_dense.bed.gz


### RNA-seq

In [ ]:
%%bash

#Get Human Bronchial Epithelial Cells (HBEC)

#Get bigwig of alignments
# Minus strand
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF730XSB_HBEC_rnaseq_minus.bw --quiet \
    "https://encode-public.s3.amazonaws.com/2016/02/24/e2d0908f-7b88-40d5-ae12-526837091e9d/ENCFF730XSB.bigWig"

# Plus strand
wget -nc -O /input_dir/corona_analysis/tracks/NCFF322WTN_HBEC_rnaseq_plus.bw --quiet \
    "https://encode-public.s3.amazonaws.com/2016/02/24/a7520a56-fc0a-450c-80c1-12d201fbd405/ENCFF322WTN.bigWig"

#Transcript quantifications
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF679UBR_HBEC_rnaseq_transcript.tsv --quiet \
    "https://encode-public.s3.amazonaws.com/2016/02/24/d293a404-1fb5-4aee-a300-b95ba3ca682c/ENCFF679UBR.tsv"
    
#Gene quantifications
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF207VFL_HBEC_rnaseq_gene.tsv --quiet \
    "https://encode-public.s3.amazonaws.com/2016/02/24/720e0bc2-c342-4542-946d-6c5728049c32/ENCFF207VFL.tsv"
    

In [5]:
%%bash

#Get NHLF Human lung fibroblast RNA-seq

#Get bigwig of alignments
# Minus strand
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF121EOD_NHLF_rnaseq_minus_hg38.bw --quiet \
    "https://encode-public.s3.amazonaws.com/2016/02/29/378b154b-29e3-45d7-8eb5-74f2a335da9a/ENCFF121EOD.bigWig"

# Plus strand
wget -nc -O /input_dir/corona_analysis/tracks/ENCFF996XPQ_NHLF_rnaseq_plus_hg38.bw --quiet \
    "https://encode-public.s3.amazonaws.com/2016/02/29/f65439f6-4b3c-495b-91a5-a571b5937d33/ENCFF996XPQ.bigWig"


# Prepare wigs from BALF of control and COVID19 patients

In [4]:
%%bash

#Get chromosomes sizes and filter to canonical ones
wget -nc --quiet http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
grep -v "_alt" hg38.chrom.sizes | grep -v "_hap" | grep -v "Un_" | grep -v "random" | sed 's/^chr//g' | sed 's/^M/MT/g' > hg38.chrom.filtered.sizes 
rm hg38.chrom.sizes


### BALF RNA-seq control

In [ ]:
%%bash

#BALF Control wigs to bigwig 
#s1
#Plus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_control_quant/BALF_control_s1_Signal.UniqueMultiple.str1.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_control_s1_rnaseq_plus_hg38.bw
#Minus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_control_quant/BALF_control_s1_Signal.UniqueMultiple.str2.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_control_s1_rnaseq_minus_hg38.bw

#s2
#Plus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_control_quant/BALF_control_s2_Signal.UniqueMultiple.str1.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_control_s2_rnaseq_plus_hg38.bw
#Minus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_control_quant/BALF_control_s2_Signal.UniqueMultiple.str2.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_control_s2_rnaseq_minus_hg38.bw

#s3
#Plus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_control_quant/BALF_control_s3_Signal.UniqueMultiple.str1.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_control_s3_rnaseq_plus_hg38.bw
#Minus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_control_quant/BALF_control_s3_Signal.UniqueMultiple.str2.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_control_s3_rnaseq_minus_hg38.bw


In [ ]:
%%bash

#BALF Infected wigs to bigwig 
#s1
#Plus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_infected_quant/BALF_corona_s1_Signal.UniqueMultiple.str1.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_corona_s1_rnaseq_plus_hg38.bw
#Minus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_infected_quant/BALF_corona_s1_Signal.UniqueMultiple.str2.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_corona_s1_rnaseq_minus_hg38.bw
   
#s2
#Plus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_infected_quant/BALF_corona_s2_Signal.UniqueMultiple.str1.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_corona_s2_rnaseq_plus_hg38.bw
#Minus strand
wigToBigWig /input_dir/corona_analysis/alignment_out/BALF_infected_quant/BALF_corona_s2_Signal.UniqueMultiple.str2.out.wig \
    hg38.chrom.filtered.sizes \
    /input_dir/corona_analysis/tracks/BALF_corona_s2_rnaseq_minus_hg38.bw


### Flip BALF bw to hg19

In [ ]:
%%bash

#Control BALF S1
CrossMap.py bigwig /input_dir/corona_analysis/annotations/hg38ToHg19.over.chain.gz /input_dir/corona_analysis/tracks/BALF_control_s1_rnaseq_plus_hg38.bw \
    /input_dir/corona_analysis/tracks/BALF_control_s1_rnaseq_plus_hg19
CrossMap.py bigwig /input_dir/corona_analysis/annotations/hg38ToHg19.over.chain.gz /input_dir/corona_analysis/tracks/BALF_control_s1_rnaseq_minus_hg38.bw \
    /input_dir/corona_analysis/tracks/BALF_control_s1_rnaseq_minus_hg19

#Infected BALF S1
CrossMap.py bigwig /input_dir/corona_analysis/annotations/hg38ToHg19.over.chain.gz /input_dir/corona_analysis/tracks/BALF_corona_s1_rnaseq_plus_hg38.bw \
    /input_dir/corona_analysis/tracks/BALF_corona_s1_rnaseq_plus_hg19
CrossMap.py bigwig /input_dir/corona_analysis/annotations/hg38ToHg19.over.chain.gz /input_dir/corona_analysis/tracks/BALF_corona_s1_rnaseq_minus_hg38.bw \
    /input_dir/corona_analysis/tracks/BALF_corona_s1_rnaseq_minus_hg19


### eQTLs

### Mendelian diseases

In [ ]:
%%bash

#Get mendelian variation tied to disease using clinvar hg38
wget --quiet -nc -O disease_names_clinvar.txt ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/disease_names


In [ ]:
%%bash

wget --quiet -nc -O clinvar_curr_hg38.vcf.gz ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar_20200615.vcf.gz
gunzip clinvar_curr_hg38.vcf.gz


In [ ]:
%%bash

wget --quiet -nc -O clinvar_curr_hg19.vcf.gz ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar_20200615.vcf.gz
gunzip clinvar_curr_hg19.vcf.gz


In [ ]:
%%bash

#get only lung related disease ConceptIDs
grep -i "lung\|COPD" disease_names_clinvar.txt | cut -f 3 | sort -u | sed '/^$/d' > lung_names.txt


In [ ]:
import vcf

disease_ids = set()

with open('lung_names.txt', 'r') as diseases:
    for line in diseases.readlines():
        disease_ids.add(line.strip())

vcf_reader = vcf.Reader(open('clinvar_curr_hg19.vcf', 'r',encoding='utf-8'))

bed_lung_var_out = open("/input_dir/corona_analysis/annotations/clinvar_kidney_variants_hg19.bed","w")

for record in vcf_reader:
    record_keys = record.INFO.keys()
    if ("CLNDISDB" in record_keys and record.INFO["CLNDISDB"][0] is not None and "MedGen" in record.INFO["CLNDISDB"][0]):
        cur_id = (record.INFO["CLNDISDB"][0]).split(":")[1]
        cur_disease = (record.INFO["CLNDN"][0])
        cur_rs = "NA"
        if "RS" in record.INFO.keys():
            cur_rs = "rs"+record.INFO["RS"][0]
        
        if cur_id in disease_ids:
            out_record = ("chr" + str(record.CHROM) +
                "\t" + str(record.start) + 
                "\t" + str(record.end) + 
                "\t" + cur_disease + 
                ":" + cur_rs + 
                "\t" + "0" + 
                "\t" + "." + "\n")
            bed_lung_var_out.write(out_record)
            
bed_lung_var_out.close()

### Methylation

## GWAS CoVid

In [ ]:
%%bash

#Get GWAS from https://www.medrxiv.org/content/10.1101/2020.05.31.20114991v1
#Got regions from supplementary table 2
# saved in /input_dir/corona_analysis/tracks/Ellinghaus_covid_gwas_hg38.bed


In [ ]:
%%bash

CrossMap.py bed /input_dir/corona_analysis/annotations/hg38ToHg19.over.chain.gz /input_dir/corona_analysis/tracks/Ellinghaus_covid_gwas_hg38.bed \
    Ellinghaus_covid_gwas_hg19.bed



In [ ]:
#Flip output to hg19
liftover_bed(from_genome="hg38",
            to_genome="hg19",
            in_bed="/input_dir/corona_analysis/tracks/Ellinghaus_covid_gwas_hg38.bed",
            out_bed="/input_dir/corona_analysis/tracks/Ellinghaus_covid_gwas_hg19.bed")


In [ ]:
%%bash

#Get GWAS from https://www.covid19hg.org ANA5 susceptibility 
wget -nc --quiet -O /input_dir/corona_analysis/gwas/covid19hg/COVID19_HGI_ANA5_20200513.txt.gz --no-check-certificate \
    https://storage.googleapis.com/covid19-hg-public/20200508/results/COVID19_HGI_ANA5_20200513.txt.gz
    
gunzip -f /input_dir/corona_analysis/gwas/covid19hg/COVID19_HGI_ANA5_20200513.txt.gz


In [ ]:

#Generate filtered, sorted bed file of variants above threshold
thresh = 1e-6
input_bed = "/input_dir/corona_analysis/gwas/covid19hg/COVID19_HGI_ANA5_20200513.txt"
out_bed = "/input_dir/corona_analysis/tracks/covid_hg19_1e6_susceptibility.bed"
filter_gwas_2_bed(input_bed=input_bed, 
                  out_bed=out_bed, 
                  col_ix=[0,1,4,18], 
                  header=True, 
                  thresh=thresh, 
                  delim='\t')


In [ ]:
#Flip output to hg19
liftover_bed(from_genome="hg38",
            to_genome="hg19",
            in_bed="/input_dir/corona_analysis/tracks/covid_hg19_1e6_susceptibility.bed",
            out_bed="/input_dir/corona_analysis/tracks/covid_hg38_1e6_susceptibility.bed")


In [ ]:
%%bash

#Generate target regions by merging thresholded p-values - 10/19
#hg19
bedops --range 50000 -m "/input_dir/corona_analysis/tracks/covid_hg19_1e6_susceptibility.bed" \
    | sort-bed - > "/input_dir/corona_analysis/tracks/covid_hg19_1e6_susceptibility_50kbMerged.bed"


In [ ]:
%%bash

#Generate target regions by merging thresholded p-values - 10/19
#hg38
bedops --range 50000 -m "/input_dir/corona_analysis/tracks/covid_hg38_1e6_susceptibility.bed" \
    | sort-bed - > "/input_dir/corona_analysis/tracks/covid_hg38_1e6_susceptibility_50kbMerged.bed"


In [ ]:
%%bash

#Pad wide for wide look on loci

#hg38
bedops --range 500000 --everything /input_dir/corona_analysis/tracks/covid_hg38_1e6_susceptibility_50kbMerged.bed \
    | sort-bed - > /input_dir/corona_analysis/tracks/covid_hg38_1e6_susceptibility_500kbMerged.bed



In [ ]:
%%bash

#Get GWAS from https://www.covid19hg.org ANA2 hospitalization 
wget -nc --quiet -O /input_dir/corona_analysis/gwas/covid19hg/COVID19_HGI_ANA2_20200513.txt.gz --no-check-certificate \
    https://storage.googleapis.com/covid19-hg-public/20200508/results/COVID19_HGI_ANA2_20200513.txt.gz
    
gunzip -f /input_dir/corona_analysis/gwas/covid19hg/COVID19_HGI_ANA2_20200513.txt.gz


In [ ]:

#Generate filtered, sorted bed file of variants above threshold
thresh = 1e-6
input_bed = "/input_dir/corona_analysis/gwas/covid19hg/COVID19_HGI_ANA2_20200513.txt"
out_bed = "/input_dir/corona_analysis/tracks/covid_hg19_1e6_hospitalization.bed"
filter_gwas_2_bed(input_bed=input_bed, 
                  out_bed=out_bed, 
                  col_ix=[0,1,4,13], 
                  header=True, 
                  thresh=thresh, 
                  delim='\t')


In [ ]:
#Flip output to hg19
liftover_bed(from_genome="hg38",
            to_genome="hg19",
            in_bed="/input_dir/corona_analysis/tracks/covid_hg19_1e6_hospitalization.bed",
            out_bed="/input_dir/corona_analysis/tracks/covid_hg38_1e6_hospitalization.bed")


In [ ]:
%%bash

#Generate target regions by merging thresholded p-values - 10/19
#hg19
bedops --range 50000 -m "/input_dir/corona_analysis/tracks/covid_hg19_1e6_hospitalization.bed" \
    | sort-bed - > "/input_dir/corona_analysis/tracks/covid_hg19_1e6_hospitalization_50kbMerged.bed"


In [ ]:
%%bash

#Generate target regions by merging thresholded p-values - 10/19
#hg38
bedops --range 50000 -m "/input_dir/corona_analysis/tracks/covid_hg38_1e6_hospitalization.bed" \
    | sort-bed - > "/input_dir/corona_analysis/tracks/covid_hg38_1e6_hospitalization_50kbMerged.bed"
